# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

### Chroma

In [19]:
# import sys
# sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

from langchain.vectorstores.chroma import Chroma
from dotenv import load_dotenv, find_dotenv
import os
# # 从环境变量中加载你的 API_KEY
# _ = load_dotenv(find_dotenv())    # read local .env file
# zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# 定义持久化目录
persist_directory = '../data_base/vector_db/chroma-vmax'

# # 创建嵌入模型
# from langchain_community.embeddings import ZhipuAIEmbeddings

# zhipu_embed = ZhipuAIEmbeddings(
#     model="embedding-2",
#     api_key=zhipuai_api_key
# )

from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://localhost:11434',model="bge-m3:latest")

try:
    # 加载持久化的 Chroma 向量数据库
    vectordb = Chroma(
        persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
        collection_name="vmax-s",
        embedding_function=my_emb
    )
    print("向量数据库已成功加载。")
except Exception as e:
    print(f"加载向量数据库时发生错误: {e}")

向量数据库已成功加载。


In [3]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：891


In [4]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：891


In [5]:
question = "VMAX上网日志业务是什么？"
docs = vectordb.similarity_search(question,k=5)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：5


打印一下检索到的内容

In [6]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第0个内容: 
 4 功能本章包含如下主题：上网日志保存
10
上网日志查询
10
上网日志批量导入查询
11
日志管理
11
账号管理
11
角色管理
12
资源监控
12
告警管理
12
省级网关对接
12
拨测结果自动比对功能
12
NAT日志入库功能
13
北向接口
13
上网日志历史查询
13
云化上网日志XDR查询
13
以下介绍ZXVMAX-S的主要的功能。
4.1 上网日志保存支持使用Gbase数据库或HDFS保存上网日志。上网日志保存时间可配置，最短保存7天时间，最长可保存一年时间。支持自动清理超过保存时间的上网日志。
4.2 上网日志查询可通过web界面指定查询条件，查询用户上网日志。支持的查询条件：时间范围+公网IP
时间范围+目的IP
时间范围+MSISDN
时间范围+IMSI
时间范围+URL
支持组合以上五种基础条件，或在基础条件上增加指定公网端口、目的端口，做更精确的查询。
10
SJ-20220623151803-017|2023-03-30（R1.0）
-----------------------------------------------------
检索到的第1个内容: 
 4功能
4.11 NAT日志入库功能
ZXVMAX-S支持将NAT日志保存到GP数据库，可通过公网IP或目的IP查询NAT日志。
4.12 北向接口支持中移规范北向接口，中移集团查询平台下发查询命令，上网日志系统将查询结果以文件方式上传至查询平台。包括云化核心网的4G/5G上网日志查询、4G/5G控制面用户面XDR查询、
4G/5G软采XDR查询、VoLTEXDR查询、5G消息XDR查询，以及原始码流查询功能。
4.13 上网日志历史查询当现网使用两个库分别存储2个月数据和6个月数据时，使用历史数据查询操作查询6个月库。
4.14 云化上网日志XDR查询上网日志系统支持云化上网日志XDR查询，包括云化核心网的4G/5G控制面用户面XDR查询、
4G/5G软采XDR查询、VoLTEXDR查询、5G消息XDR查询，以及原始码流查询功能。
SJ-20220623151803-017|2023-03-30（R1.0）
13
------------------------------------------

### Milvus

In [8]:
from langchain_community.vectorstores import Milvus
my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )

In [10]:
results = vectordb.similarity_search(query="VMAX", k=5)
results

[Document(metadata={'producer': 'Apache FOP Version 2.6', 'creator': 'DITA Open Toolkit', 'creationdate': '2023-05-23T21:44:01+08:00', 'source': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（语音业务）.pdf', 'file_path': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（语音业务）.pdf', 'total_pages': 41, 'format': 'PDF 1.4', 'title': '目录', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20230523214401+08'00'", 'page': 16, 'pk': 457237404597884273}, page_content='4\xa0功能与业务本章包含如下主题：\uf06c功能\n13\n\uf06c业务\n16\n4.1\xa0功能本节介绍ZXVMAX-S语音业务的主要的功能。\n4.1.1\xa0用户分析功能用户分析是以客户、客户群组等为维度的专项分析，可以区分业务，区分用户等级，区分时间等，进行用户级的在某一时间段的呼叫、注册记录，媒体数据、信令详情，并结合该用户在最近的呼叫、注册统计情况，分析定位语音/视频通话质量发生异常的环节。由于网元众多，运营维护人员在发生用户投诉后难以定位具体问题所在，事后拨测定位，在各个接口上同时抓包，工作量繁重。用户分析功能提供了运营维护人员以下帮助：\uf06c问题初步定位根据用户IMSI或者号码，查询在某一时间段的呼叫、注册记录，媒体数据、信令详情，并结合该用户在最近7天的呼叫、注册统计情况，分析定位语音/视频通话质量发生异常的环节。\uf06c详细分析可以更进一步钻取可以根据用户IMSI或者号码，回溯VMAX系统接入采集的所有信令，展现信令流程图，无需事后拨测抓包。'),
 Document(m

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [ ]:
# # 从环境变量中加载你的 API_KEY
# _ = load_dotenv(find_dotenv())    # read local .env file
# zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# from langchain_openai import ChatOpenAI
# zhipuai_llm = ChatOpenAI(
#     temperature=0,
#     model="glm-4",
#     openai_api_key=zhipuai_api_key,
#     openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
# )

In [29]:
from langchain_community.llms import Ollama

my_llm = Ollama(base_url='http://localhost:11434', model='deepseek-r1:14b', temperature=0.1)

my_llm.invoke("nihao")

'<think>\n\n</think>\n\n你好！有什么我可以帮助你的吗？'

In [ ]:
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain_ollama import OllamaLLM

# my_llm = OllamaLLM(base_url='http://129.201.70.35:11434', model='deepseek-r1:14b', temperature=0.1, streaming=True,
#     callbacks=[StreamingStdOutCallbackHandler()])

# my_llm.invoke("nihao")

In [34]:
# from langchain_deepseek import ChatDeepSeek
# _ = load_dotenv(find_dotenv())    # read local .env file
# deepseek_api_key = os.environ['DEEPSEEK_API_KEY']
# llm_deepseek = ChatDeepSeek(
#     model="deepseek-reasoner",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     # max_retries=2,
#     api_key=deepseek_api_key,
#     # other params...
# )

In [35]:
# from langchain_deepseek import ChatDeepSeek

# _ = load_dotenv(find_dotenv())    # read local .env file
# deepseek_api_key = os.environ['DEEPSEEK_API_KEY']

# from langchain_openai import ChatOpenAI
# zhipuai_llm = ChatOpenAI(
#     temperature=0,
#     model="deepseek-chat",
#     openai_api_key=deepseek_api_key,
#     openai_api_base="https://api.deepseek.com"
# )

## 3. 构建检索问答链

prompts

In [30]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [31]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(my_llm,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


In [32]:
question_1 = "什么是vmax？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
<think>
好的，我现在要回答用户的问题：“什么是VMAX？”根据提供的上下文资料，我需要先理解相关内容，然后整理出一个清晰的解释。

首先，资料中多次提到“ZXVMAX-S多维价值分析系统”。这表明VMAX可能是指这个系统。从内容来看，该系统涉及上网日志保存、查询、管理，以及使用Gbase数据库和HDFS存储数据。此外，还提到了故障处理流程和告警机制。

接下来，资料中详细描述了系统的功能模块，如上网日志保存、查询、批量导入、日志管理等，说明VMAX是一个用于管理和分析网络日志的系统。它可能用于电信或互联网行业，帮助监控和维护网络性能，检测异常流量，并提供数据挖掘和统计分析功能。

另外，资料中还提到了硬件结构部分，说明该系统基于标准IT部件构建，适用于处理海量数据。同时，故障处理流程显示了系统的稳定性和技术支持的重要性。

综合以上信息，VMAX很可能是一个专业的网络日志管理平台，用于收集、存储、分析和监控网络活动，帮助管理员进行故障排查、性能优化和安全审计。
</think>

**VMAX** 是指 **ZXVMAX-S 多维价值分析系统**，这是一个专业的网络日志管理平台。该系统主要用于电信或互联网行业，能够高效地处理海量数据，包括上网日志的保存、查询、批量导入以及数据分析等功能。它支持使用 Gbase 数据库和 HDFS 进行存储，并提供灵活的时间配置和自动清理功能。

VMAX 系统具备以下特点：
1. **日志管理**：支持通过 Web 界面进行精确查询，可组合多种条件如时间范围、IP、MSISDN、IMSI 和 URL 等。
2. **数据存储**：采用 Gbase 数据库和 HDFS，确保数据的高效存储与管理。
3. **故障处理**：提供详细的故障处理流程和告警机制，帮助快速定位和解决系统问题。
4. **硬件支持**：基于标准 IT 部件构建，适用于大规模部署。

总之，VMAX 是一个功能强大的网络日志分析系统，旨在帮助管理员进行网络监控、性能优化和安全审计。


In [33]:
question_1 = "严威是谁？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
<think>

</think>

对不起，我还没有学会回答这个问题。如果你有其他问题，我非常乐意为你提供帮助。


### 创建一个基于模板的检索链： rerank检索版本

#### cohere

In [ ]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
# rerank检索
# Cohere Rerank配置
import cohere
cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")

compressor = CohereRerank(
    client=cohere_client,
    top_n=5,
    model="rerank-multilingual-v3.0"  # 支持多语言的新版本
)

base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)


qa_chain = RetrievalQA.from_chain_type(
    llm_deepseek,
    retriever=compression_retriever,  # 替换为压缩检索器
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        # "llm_kwargs": {"max_length": 300}  # 新增输出长度限制
    }
)


#### jina

In [35]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import JinaRerank  # 使用Jina的rerank组件

# Jina Rerank配置
JINA_API_KEY = "jina_63bb115e2d5f42d581f42643294792b5CE4nrEINMDcT4vJZJaSLcr5tkbIB"  # 替换为你的Jina API密钥

compressor = JinaRerank(
    jina_api_key=JINA_API_KEY,
    top_n=3,
    model="jina-reranker-v2-base-multilingual"  # Jina的多语言rerank模型[5](@ref)
)

base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)


qa_chain = RetrievalQA.from_chain_type(
    my_llm,
    retriever=compression_retriever,  # 替换为压缩检索器
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        # "llm_kwargs": {"max_length": 300}  # 新增输出长度限制
    }
)

#### BGE版本

In [ ]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import JinaRerank  # 使用Jina的rerank组件

# BGE配置
# 先将模型下载到本地
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="/opt/workspace/models/BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=model, top_n=3)

base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)

compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)


qa_chain = RetrievalQA.from_chain_type(
    my_llm,
    retriever=compression_retriever,  # 替换为压缩检索器
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        # "llm_kwargs": {"max_length": 300}  # 新增输出长度限制
    }
)

## 4.检索问答链效果测试

In [37]:
question_1 = "什么是VMAX-S的上网日志业务？"
question_2 = "严威是谁？"

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [38]:
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
<think>
好的，我现在需要回答用户的问题：“什么是VMAX-S的上网日志业务？”首先，我得仔细阅读提供的上下文，找出相关的信息。

从产品描述中可以看到，ZXVMAX-S多维价值分析系统专注于上网日志业务。它有几个主要模块：采集层、存储共享层和应用分析层。采集层负责收集信令数据和防火墙日志，并将它们合并生成上网日志。存储层使用分布式集群和MPP数据库来保存这些日志，同时还有一个Saturn模块处理实时流数据。

接下来是应用分析层，它提供了客户端访问接口、RESTful API以及服务器端的逻辑和算法，用于分析这些日志数据。此外，系统还具备角色管理功能，提供四种不同的角色权限，确保系统的安全性。还有资源监控和告警管理，帮助运维人员及时发现和处理问题。

总结一下，VMAX-S的上网日志业务主要是通过采集、存储和分析网络日志数据，来实现对网络行为的监控和管理，并提供了相应的安全措施和技术支持。
</think>

ZXVMAX-S多维价值分析系统的产品描述中提到，该系统专注于“上网日志业务”。具体来说，它通过采集层（包括探针、防火墙日志处理单元和数据合成单元）收集信令数据和防火墙日志，并生成上网日志数据。存储共享层使用分布式存储集群和MPP数据库Gbase来保存这些日志记录，同时Saturn模块负责实时流数据的接收和解析。

应用分析层则通过客户端、REST接口和服务器端逻辑提供对日志数据的访问和分析功能。此外，系统还具备角色管理、资源监控和告警管理等功能，以确保系统的安全性和稳定性。

因此，ZXVMAX-S的上网日志业务主要是指该系统在收集、存储和分析网络日志数据方面的功能，以及相关的安全管理与监控能力。

谢谢你的提问！


In [39]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
<think>
好的，我现在需要回答用户的问题：“严威是谁？”首先，我查看了提供的上下文内容。上下文主要介绍了ZXVMAX-S端到端系统及其功能、告警处理和产品描述，但没有提到任何关于“严威”的信息。

接下来，我分析用户可能的意图。用户可能在寻找某个特定人物的信息，比如公司员工、公众人物或技术专家。然而，在提供的资料中，并没有任何关于个人的内容，只有产品和技术细节。

考虑到上下文内容与问题无关，我无法找到任何相关的信息来回答这个问题。因此，根据指示，如果不知道答案，应该明确表示不知道，并避免编造信息。

最后，按照要求，回答结束后需要加上“谢谢你的提问！”。
</think>

严威不是在提供的上下文中提到的人物，因此我不知道严威是谁。谢谢你的提问！


### 4.2 大模型自己回答的效果

In [40]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
my_llm.predict(prompt_template)

/tmp/ipykernel_41296/46941173.py:5: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  my_llm.predict(prompt_template)


'<think>\n嗯，我现在要了解什么是VMAX-S的上网日志业务。首先，我听说过VMAX这个品牌，可能是一个网络设备或者服务提供商。S可能代表的是“安全”或者其他含义，但我不确定。\n\n上网日志业务听起来像是记录用户上网活动的服务。那具体来说，VMAX-S的这个业务是做什么的呢？是不是用来监控用户的网络使用情况，比如访问了哪些网站，登录了什么应用，或者记录下了线时间？\n\n我记得以前听说过企业或学校会用类似的东西来管理内部网络，确保员工或学生遵守规定，不访问不良网站。所以，VMAX-S可能提供这样的日志记录功能，帮助机构监控和管理网络使用。\n\n那这个业务具体包括哪些内容呢？可能有详细的上网记录，比如IP地址、MAC地址这些信息，还有用户访问的URL，登录的应用系统，以及上下线时间等。这些数据可以帮助管理员了解用户的网络行为，进行审计或者安全分析。\n\n另外，日志存储和查询功能也很重要。VMAX-S可能会提供一个平台或工具，让用户可以方便地查看和管理这些日志数据，可能还会生成报告，帮助机构更好地理解和处理网络活动。\n\n合规性也是一个考虑因素。很多行业有法规要求记录用户的上网行为，比如金融、教育或者政府机构。VMAX-S的这个业务可能帮助企业满足这些合规要求，避免法律风险。\n\n总的来说，VMAX-S的上网日志业务应该是提供一个全面的日志记录和管理解决方案，帮助企业和机构监控网络使用情况，确保安全和合规性。\n</think>\n\nVMAX-S的上网日志业务是一种由VMAX提供的网络管理服务，主要用于记录和分析用户的上网行为。该业务通过收集详细的网络活动数据，如IP地址、MAC地址、访问URL、应用登录信息及上下线时间等，帮助企业和机构监控内部网络使用情况，确保符合安全策略和法规要求。其主要功能包括全面的日志记录、存储与查询、实时监控告警以及生成审计报告，适用于金融、教育、政府等多个行业，以满足合规需求并保障网络安全。'

In [41]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
my_llm.predict(prompt_template)

'<think>\n\n</think>\n\n对不起，我还没有学会回答这个问题。如果你有其他问题，我非常乐意为你提供帮助。'

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能-v2025

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习 Python 吗？”，并查看回答。

v2024

In [ ]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm_deepseek,
    retriever=retriever,
    memory=memory
)
question = "什么是VMAX-S的上网日志业务？"
result = qa({"question": question})
print(result['answer'])

V2025

不带memory

In [ ]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
# rerank检索

# Jina Rerank配置
JINA_API_KEY = "jina_63bb115e2d5f42d581f42643294792b5CE4nrEINMDcT4vJZJaSLcr5tkbIB"  # 替换为你的Jina API密钥

compressor = JinaRerank(
    jina_api_key=JINA_API_KEY,
    top_n=3,
    model="jina-reranker-v2-base-multilingual"  # Jina的多语言rerank模型[5](@ref)
)


# # BGE配置
# # 先将模型下载到本地
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import CrossEncoderReranker
# from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# model = HuggingFaceCrossEncoder(model_name="/opt/workspace/models/BAAI/bge-reranker-base")
# compressor = CrossEncoderReranker(model=model, top_n=3)

base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)


qa_chain = RetrievalQA.from_chain_type(
    my_llm,
    retriever=compression_retriever,  # 替换为压缩检索器
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        # "llm_kwargs": {"max_length": 300}  # 新增输出长度限制
    }
)

result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])


新增MEMORY

In [ ]:
from langchain.chains import ConversationalRetrievalChain

from langchain.prompts import PromptTemplate

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["chat_history", "question", "context"],
    template="""
    你是一个专业的问答助手。请根据对话历史和提供的上下文回答问题。
    
    历史对话：
    {chat_history}
    
    上下文：
    {context}
    
    问题：{question}
    
    回答：
    """
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm_deepseek,
    retriever=compression_retriever,
    memory=memory,
    # return_source_documents=True,
    output_key="answer",  # 明确指定存储到内存的键
    combine_docs_chain_kwargs={  # 替代chain_type_kwargs
        "prompt": QA_CHAIN_PROMPT
    },
    verbose=True, # 独立传递verbose参数
)


questions = [
    "什么是VMAX的安全加固？", 
    "安全加固的操作步骤？",  # 需记忆前一轮的"主要内容"
    "整理成中文表格"  # 需合并多轮信息
]

for question in questions:
    result = qa_chain({"question": question})
    print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    # print(f"引用的来源：{result['source_documents'][0].metadata}")  # 显示来源文档
    print("对话历史：", memory.load_memory_variables({}))

## 5. 添加历史对话的记忆功能 - v2024

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习 Python 吗？”，并查看回答。

In [ ]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
question = "什么是VMAX？"
result = qa({"question": question})
print(result['answer'])

然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [ ]:
question = "南瓜书包含哪些功能？"
result = qa({"question": question})
print(result['answer'])

可以看到，LLM 它准确地判断了这方面的知识，指代内容是强化学习的知识，也就
是我们成功地传递给了它历史信息。这种持续学习和关联前后问题的能力，可大大增强问答系统的连续
性和智能水平。

# 脚本汇总

In [ ]:
def get_llm():
    return OllamaLLM(base_url='http://localhost:11434', model='deepseek-r1:14b', temperature=0.1)


def generate_response(input_text):
    llm = get_llm()
    output = llm.invoke(input_text)
    output_parser = StrOutputParser()
    return output_parser.invoke(output)

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
import cohere
from langchain_community.vectorstores import Milvus



# 初始化 Milvus 向量数据库
def get_vectordb():
    my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")
    
    # Milvus 连接参数
    vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",      # Milvus 默认端口
        },
        # 可选：是否自动创建集合（如果不存在）
        # auto_create_collection=True,
    )
    return vectordb


def get_llm():
    return OllamaLLM(base_url='http://localhost:11434', model='deepseek-r1:14b', temperature=0.1, streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()])


def get_qa_chain_without_memory(question: str):
    vectordb = get_vectordb()
    myllm = get_llm()

    # Jina Rerank配置
    JINA_API_KEY = "jina_63bb115e2d5f42d581f42643294792b5CE4nrEINMDcT4vJZJaSLcr5tkbIB"  # 替换为你的Jina API密钥
    
    compressor = JinaRerank(
        jina_api_key=JINA_API_KEY,
        top_n=3,
        model="jina-reranker-v2-base-multilingual"  # Jina的多语言rerank模型[5](@ref)
    )

    base_retriever = vectordb.as_retriever(
        search_kwargs={"k": 15},
        search_type="mmr",
    )

    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=myllm,
        retriever=compression_retriever,
        return_source_documents=True,
        chain_type_kwargs={
            "prompt": PromptTemplate(
                input_variables=["context", "question"],
                template="""你是DeepSeek VMAX-S知识助手。使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
                案。总是在回答的最后说“谢谢你的提问！”。
                {context}
                问题: {question}
                """
            ),
        }
    )

    result = qa_chain({"query": question})
    return result["result"]


print(get_qa_chain_without_memory("介绍下VMAX的上网日志业务"))


In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
import cohere

# Initialize memory outside the function so it persists across questions
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# 初始化 Milvus 向量数据库
def get_vectordb():
    my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")
    
    # Milvus 连接参数
    vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "192.168.0.188",  # Milvus 服务器地址
            "port": "19530",      # Milvus 默认端口
        },
        # 可选：是否自动创建集合（如果不存在）
        # auto_create_collection=True,
    )
    return vectordb

def get_llm():
    return OllamaLLM(base_url='http://localhost:11434', model='deepseek-r1:1.5b', temperature=0.1)

def get_qa_chain_with_memory(question: str):
    vectordb = get_vectordb()
    myllm = get_llm()

    # Jina Rerank配置
    JINA_API_KEY = "jina_63bb115e2d5f42d581f42643294792b5CE4nrEINMDcT4vJZJaSLcr5tkbIB"  # 替换为你的Jina API密钥
    
    compressor = JinaRerank(
        jina_api_key=JINA_API_KEY,
        top_n=3,
        model="jina-reranker-v2-base-multilingual"  # Jina的多语言rerank模型[5](@ref)
    )
    

    base_retriever = vectordb.as_retriever(
        search_kwargs={"k": 15},
        search_type="mmr",
    )

    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever
    )

    QA_CHAIN_PROMPT = PromptTemplate(
        input_variables=["chat_history", "question", "context"],
        template="""
        你是DeepSeek VMAX-S知识助手。使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！
        
        历史对话：
        {chat_history}
        
        上下文：
        {context}
        
        问题：{question}
        
        回答：
        """
    )

    # QA_CHAIN_PROMPT = PromptTemplate(
    #     input_variables=["context","question"],
    #     template="""你是DeepSeek VMAX-S知识助手。使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
    #             {context}
    #             问题: {question}
    #     """
    # )

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=get_llm(),
        retriever=compression_retriever,
        memory=memory,
        output_key="answer",
        combine_docs_chain_kwargs={
            "prompt": QA_CHAIN_PROMPT
        },
        verbose=True,
    )

    result = qa_chain({"question": question})  # Changed from "query" to "question"
    return result

questions = [
    "什么是VMAX的上网日志业务？",
    "上网日志业务包含哪些功能？",  # 需记忆前一轮的"主要内容"
    "整理成中文表格"  # 需合并多轮信息
]

for question in questions:
    result = get_qa_chain_with_memory(question)  # Pass string directly, not dict
    print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "="*50 + "\n")

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
import cohere
from langchain_community.vectorstores import Milvus
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Singleton pattern for vector database
_vectordb_instance = None

def get_vectordb():
    global _vectordb_instance
    if _vectordb_instance is None:
        try:
            my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")
            
            _vectordb_instance = Milvus(
                embedding_function=my_emb,
                collection_name="Vmaxs",
                connection_args={
                    "host": "192.168.0.188",
                    "port": "19530",
                },
                # Add consistency level if needed
                consistency_level="Strong",
            )
            logger.info("Successfully connected to Milvus vector database")
        except Exception as e:
            logger.error(f"Failed to connect to Milvus: {str(e)}")
            raise
    return _vectordb_instance

def get_llm():
    return OllamaLLM(
        base_url='http://localhost:11434', 
        model='deepseek-r1:1.5b', 
        temperature=0.1,
        # Add timeout parameters
        request_timeout=60
    )

def get_qa_chain_without_memory(question: str):
    try:
        vectordb = get_vectordb()
        myllm = get_llm()

    # Jina Rerank配置
    JINA_API_KEY = "jina_63bb115e2d5f42d581f42643294792b5CE4nrEINMDcT4vJZJaSLcr5tkbIB"  # 替换为你的Jina API密钥
    
    compressor = JinaRerank(
        jina_api_key=JINA_API_KEY,
        top_n=3,
        model="jina-reranker-v2-base-multilingual"  # Jina的多语言rerank模型[5](@ref)
    )


        # Milvus-specific search parameters
        base_retriever = vectordb.as_retriever(
            search_kwargs={
                "k": 15,
                # Add Milvus-specific search parameters if needed
                "params": {"nprobe": 10}
            },
            search_type="mmr",
        )

        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor,
            base_retriever=base_retriever
        )

        qa_chain = RetrievalQA.from_chain_type(
            llm=myllm,
            retriever=compression_retriever,
            return_source_documents=True,
            chain_type_kwargs={
                "prompt": PromptTemplate(
                    input_variables=["context", "question"],
                    template="""你是DeepSeek VMAX-S知识助手。使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
                    案。总是在回答的最后说"谢谢你的提问！"。
                    {context}
                    问题: {question}
                    """
                ),
            }
        )

        result = qa_chain({"query": question})
        return result["result"]
    except Exception as e:
        logger.error(f"Error in QA chain: {str(e)}")
        return "抱歉，处理您的请求时出现错误。"

# Example usage
if __name__ == "__main__":
    print(get_qa_chain_without_memory("介绍下VMAX的上网日志业务"))